In [1]:
import langchain
from openai import OpenAI
import json
from datetime import datetime,timedelta

In [4]:
stu_info = "parth, 20 yo, IIIT Nagpur"

In [5]:
prompt = f'''get the data
name
age
college
from {stu_info}
in json format
'''

In [6]:
openaiKey = "ok"

In [ ]:
client  = OpenAI(api_key=openaiKey)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "user", "content": prompt}
  ]
)
# this is a few shot prompt

In [ ]:
op = response.choices[0].message.content

In [ ]:
json.loads(op)

In [ ]:
student_info = [{
    
        "name": "extract stu_info",
        "description": "Get the student information from the given text",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Name of the student"
                },
                "age": {
                    "type": "integer",
                    "description": "Age of the student"
                },
                "college": {
                    "type": "string",
                    "description": "College of the student"
                }
            }
        }
}]

In [ ]:
response2 = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "user", "content": prompt}
  ],
  functions=student_info
)

In [ ]:
json.loads(response2.choices[0].message.function_call.arguments)

In [7]:
student_description_two="krish naik is a student of computer science at IIT Mumbai. He is an indian and has a 9.5 GPA. krish is known for his programming skills and is an active member of the college's data science Club. He hopes to pursue a career in artificial intelligence after graduating."

In [8]:
student_description_three="sudhanshu kumar is a student of computer science at IIT bengalore. He is an indian and has a 9.2 GPA. krish is known for his programming skills and is an active member of the college's MLops Club. He hopes to pursue a career in artificial intelligence after graduating."

In [ ]:
student_info = [stu_info, student_description_two,student_description_three]
for student in student_info:
    response =  client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': prompt}],
        functions = student_info,
        function_call = 'auto'
    )

    response = json.loads(response.choices[0].message.function_call.arguments)
    print(response)

In [9]:
funtion_two=student_custom_function = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'college': {
                    'type': 'string',
                    'description': 'The college name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'CGPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'college club for extracurricular activities. '
                }
                
            }
        }
    }
]

In [ ]:
functions = [student_info[0], funtion_two[0]]
student_info = [stu_info, student_description_two,student_description_three]
for student in student_info:
    response =  client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': student}],
        functions = functions,
        function_call = 'auto'
    )

    response = json.loads(response.choices[0].message.function_call.arguments)
    print(response)

advanced eg of function calling

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {
      "role": "user",
      "content": "When's the next flight from delhi to mumbai?"
    }
      ]
)

In [ ]:
function_descriptions = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport, e.g. DEL",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport, e.g. MUM",
                },
            },
            "required": ["loc_origin", "loc_destination"], #required added
        },
    }
]

In [ ]:
user_prompt = "When's the next flight from new delhi to mumbai?"

In [ ]:
response2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {
      "role": "user",
      "content": user_prompt
    }
      ],
    # Add function calling
    functions=function_descriptions,
    function_call="auto",  # specify the function call
    
)

#it will return the oringin and destination from  the user prompt and then we can call any 3rd party api to extract the required information

example api

In [ ]:

def get_flight_info(loc_origin, loc_destination):
    """Get flight information between two locations."""

    # Example output returned from an API or database
    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=2)),
        "airline": "KLM",
        "flight": "KL643",
    }

    return json.dumps(flight_info)

In [ ]:
origin = json.loads(response2.choices[0].message.function_call.arguments).get("loc_origin")
destination = json.loads(response2.choices[0].message.function_call.arguments).get('loc_destination')

In [ ]:
params=json.loads(response2.choices[0].message.function_call.arguments)

In [ ]:
flight = get_flight_info(**params)
flight

<h2>so i extracted the meaningful data from a prompt and then used a 3rd party api to get the needed info</h2>

In [ ]:
response3 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {"role": "user","content": user_prompt},
    {"role": "function", "name": response2.choices[0].message.function_call.name, "content": flight}
      ],
    # Add function calling
    functions=function_descriptions,
    function_call="auto",  # specify the function call
    
)

In [ ]:
response3.choices[0].message.content